## MobileNetV2 model - ALL DATA
Creation of MobileNetV2 model for sound recognition <br>
Dataset includes Freesound, UrbanSound8k and some custom sounds generated from an iPhone <br>
<br>
Audio files converted to log-scaled MEL spectrograms. <br>
Classification model built from a headless version of MobileNetV2 from ("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"). <br>
The MobileNetV2 model was used as a feature extractor and wrapped up as a Keras layer using hub.KerasLayer and frozen so that it will not be modifiable by training.<br> <br>
A new model is created by attaching the pre-trained feature extractor model and a new classification head layer. This model is the one compiled and trained. 


Grant access to Google Drive, where the spectograms are available

In [1]:
from google.colab import drive #Only if you are using Google Drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/My\ Drive/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive
/content/gdrive/My Drive


Check contents of Tensorboard logs. Remove if necessary

In [ ]:
#! rm -R /content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit5/*
#! ls -al /content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit5/

total 0


In [2]:
!pip install tensorboardcolab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-py3-none-any.whl size=3859 sha256=d15ec2ee864877f834efa013a08cb556a75466bf106bc266130e3d53a3914525
  Stored in directory: /root/.cache/pip/wheels/69/4e/4a/1c6c267395cb10edded1050df12af165d3254cfce324e80941
Successfully built tensorboardcolab


Load necessary Python modules

In [2]:
import matplotlib.pyplot as plt
import matplotlib.pylab as plabt
import numpy as np
import os
import PIL
import time
import datetime
import PIL.Image as Image

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback


%load_ext tensorboard

In [ ]:
#file_writer = hub.summary.FileWriter('/content/gdrive/MyDrive/content/img_dir/logs/fit/', sess.graph)
#tbc=TensorBoardColab()
#summary_writer = tbc.get_writer()
#summary_writer = tf.train.SummaryWriter()
#summary_writer = tf.summary.create_file_writer("/content/gdrive/MyDrive/content/img_dir/logs/freesound/fit/")
#log_dir = "/content/gdrive/MyDrive/content/img_dir/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#%tensorboard --logdir logs_dir

Define necessary parameters for the model and location of images in Google Drive

In [3]:
batch_size = 32
img_height = 224
img_width = 224
#BATCH_SIZE = 255
IMG_SIZE = (224, 224)

#train_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/train'
#validation_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/test/'
data_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/all_labeled/'

mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

feature_extractor_model = mobilenet_v2

## Datasets
Define training and validation (test) set using Keras Utility **image_dataset_from_directory** including data augmentation. <br>

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

Found 49449 files belonging to 217 classes.
Using 39560 files for training.
Found 49449 files belonging to 217 classes.
Using 9889 files for validation.


In [5]:
class_names = np.array(train_ds.class_names)
print(class_names)

['Accelerating_and_revving_and_vroom' 'Accordion' 'Acoustic_guitar'
 'Aircraft' 'Alarm' 'Animal' 'Applause' 'Bark' 'Bass_drum' 'Bass_guitar'
 'Bathtub_filling_or_washing' 'Bell' 'Bicycle' 'Bicycle_bell' 'Bird'
 'Bird_vocalization_and_bird_call_and_bird_song' 'Boat_and_Water_vehicle'
 'Boiling' 'Boom' 'Bowed_string_instrument' 'Brass_instrument' 'Breathing'
 'Burping_and_eructation' 'Bus' 'Buzz' 'Camera' 'Car' 'Car_passing_by'
 'Cat' 'Chatter' 'Cheering' 'Chewing_and_mastication'
 'Chicken_and_rooster' 'Child_speech_and_kid_speaking' 'Chime'
 'Chink_and_clink' 'Chirp_and_tweet' 'Chuckle_and_chortle' 'Church_bell'
 'Clapping' 'Clock' 'Coin_dropping' 'Computer_keyboard' 'Conversation'
 'Cough' 'Cowbell' 'Crack' 'Crackle' 'Crash_cymbal' 'Cricket' 'Crow'
 'Crowd' 'Crumpling_and_crinkling' 'Crushing' 'Crying_and_sobbing'
 'Cupboard_open_or_close' 'Cutlery_and_silverware' 'Cymbal'
 'Dishes_and_pots_and_pans' 'Dog' 'Domestic_sounds_and_home_sounds' 'Door'
 'Doorbell' 'Drawer_open_or_close' 'Dr

In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


Create the feature extractor by wrapping the pre-trained model as a Keras layer with hub.KerasLayer. Use the trainable=False argument to freeze the variables, so that the training only modifies the new classifier layer:

In [7]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    name="MobileNetV2",
    input_shape=(224, 224, 3),
    trainable=False)

In [ ]:
#model.get_layer(index=0).summary()
#model.get_layer('keras_layer').summary()

The feature extractor returns a 1280-long vector for each image (the image batch size remains at 32 in this example):

In [8]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)


(32, 1280)


Attach a classification head

To complete the model, wrap the feature extractor layer in a tf.keras.Sequential model and add a fully-connected layer for classification

In [9]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobileNetV2 (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 217)               277977    
                                                                 
Total params: 2,535,961
Trainable params: 277,977
Non-trainable params: 2,257,984
_________________________________________________________________


In [10]:
#model.get_layer(index=0) #.summary()
mylayer=model.get_layer('MobileNetV2') #.summary()
print(mylayer)

In [11]:
predictions = model(image_batch)
predictions.shape

TensorShape([32, 217])

**Train the model**

Use Model.compile to configure the training process and add a tf.keras.callbacks.TensorBoard callback to create and store logs:

In [12]:
# changed default adam optimizer to a very low learning rate
#   optimizer=tf.keras.optimizers.Adam(1e-5),
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-3),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dest = "/content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit6/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log, histogram_freq=1) # Enable histogram computation for every epoch.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dest, write_graph=True, write_images=False, histogram_freq=1)

    

In [16]:
! rm -R /content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit6/*
! ls -lrt /content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit6

total 0


Now use the Model.fit method to train the model.

To visualize the training progress in TensorBoard later, create and store logs an a TensorBoard callback.

In [ ]:
#log = "/content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log, histogram_freq=1) # Enable histogram computation for every epoch.

In [ ]:
NUM_EPOCHS = 30

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/30
 964/1237 [======================>.......] - ETA: 1:33 - loss: 3.3856 - acc: 0.2884

In [ ]:
#model = keras.models.load_model('path/to/location')

In [ ]:
model.save('/content/gdrive/MyDrive/content/models/mobilenetv2_all_combined_headless_final5_30ep_LLR')
model.save('/content/gdrive/MyDrive/content/models/mobilenetv2_all_combined_headless_final5_30ep_LLR.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#!kill 2933
#! rm -R /content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit4/
! ls -lrt /content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit6/

In [ ]:
 #! kill -9 2453
 %reload_ext tensorboard

In [ ]:
#!kill 2893
#%reload_ext tensorboard
%tensorboard --logdir="/content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit6"
#%tensorboard --logdir=log_dest

**Check the predictions**

Obtain the ordered list of class names from the model predictions:

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

In [ ]:
#y_ = placeholder_for_labels # for eg: [1, 2, 4]
#y = mycnn(...) # for eg: [2, 2, 4]
y_ = predicted_label_batch
y = predicted_batch



confusion = tf.confusion_matrix(labels=y_, predictions=y, num_classes=class_names)

In [ ]:
#confusion_mtx = tf.math.confusion_matrix(predicted_id, predicted_batch)
#plt.figure(figsize=(10, 8))
#sns.heatmap(confusion_mtx,
#            xticklabels=commands,
#            yticklabels=commands,
#            annot=True, fmt='g')
#plt.xlabel('Prediction')
#plt.ylabel('Label')
#plt.show()

Plot **MODEL PREDICTIONS**

In [ ]:
plt.figure(figsize=(14,13))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(  model.layers))
model.summary()

Fine tuning

In [ ]:
fine_tune_epochs = 30
total_epochs =  NUM_EPOCHS + fine_tune_epochs

history_finetune = model.fit(train_ds,
                         epochs=total_epochs,
                         validation_data=val_ds,
                         initial_epoch=history.epoch[-1],
                         callbacks=tensorboard_callback)

In [ ]:
model.save('/content/gdrive/MyDrive/content/models/mobilenetv2_all_combined_headless_ftune_final5_LLR')
model.save('/content/gdrive/MyDrive/content/models/mobilenetv2_all_combined_headless_ftune_final5_LLR.h5')

# Convert the model to Tensorflow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/content/gdrive/MyDrive/content/models/mobilenetv2_all_combined_headless_ftune_final5_20ep_LLR.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
%tensorboard --logdir="/content/gdrive/MyDrive/content/img_dir/logs/all_combined/fit6"
#%tensorboard --logdir=log_dest

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = val_ds.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
#predictions = tf.nn.sigmoid(predictions)
#predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(12, 12))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

AttributeError: ignored